### Pipeline for QnA with Memory

In [1]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [2]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [3]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 
# !pip3 install lxml
# !pip3 install lxml

In [55]:
import os
import sys
import glob
import re
import importlib
import langchain
from pathlib import Path
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, TextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import helpers as h
import constants as c
import scraper as sc
importlib.reload(h)
importlib.reload(c)
importlib.reload(sc)

<module 'scraper' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/scraper.py'>

#### Environment

In [29]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [30]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### Paths

In [31]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')
merged_pdfs_path = os.path.join(os.getcwd(), '../data/pdfs_processed/merged')

#### Data

In [9]:
investorcom_pdfs = [ 
    _ for _ in glob.glob(os.path.join(pdfs_path, '*.pdf')) if 'investorcom' in _
]

In [6]:
# web_list = sc.scrape_site(
#     "https://www.economist.com/",
#     c.header_template
# )
# web_list

In [154]:
# web_list = [
#     'https://www.scientificamerican.com/article/pangaea-ultima-the-next-supercontinent-may-doom-mammals-to-far-future-extinction/',
#     'https://www.scientificamerican.com/article/ho-oleilana-a-billion-light-year-wide-bubble-of-galaxies-astounds-astronomers/',
#     'https://www.scientificamerican.com/article/dead-trees-and-a-mysterious-cosmic-explosion-reveal-bigger-quake-risk-for-seattle/',
#     'https://www.scientificamerican.com/article/streetlights-are-mysteriously-turning-purple-heres-why/',
#     'https://www.scientificamerican.com/article/to-move-fast-quantum-maze-solvers-must-forget-the-past/',
# ]

##### - Load

In [10]:
loaded_data = []
for pdf in investorcom_pdfs :
    loaded_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        

In [11]:
# loaded_data = []
# for page in web_list[:40]:
#     try:
#         loaded_data.extend(WebBaseLoader(page).load())
#     except:
#         continue

In [12]:
len(loaded_data)

7

In [13]:
print(loaded_data[5].metadata)

{'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/meta_investorcom_1.pdf'}


In [16]:
# for pdf in pdf_list :
#     try :
#         pdf_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        
#     except NameError:
#         pdf_loader = UnstructuredPDFLoader(file_path=pdf)
#         pdf_data = pdf_loader.load()

##### - Split

In [60]:
# chunk_size = c.prompt_max//c.retrieval_kwargs['k']
chunk_size = 4000
chunk_overlap = 0

In [61]:
data_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, chunk_overlap = chunk_overlap
)

In [62]:
chunked_data = data_splitter.split_documents(loaded_data)

In [64]:
for doc in chunked_data:
    doc.metadata['company']=os.path.basename(
        doc.metadata['source']).replace('_investorcom_1.pdf', ''
   )

In [77]:
chunked_data[0].metadata

{'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/amazon_investorcom_1.pdf',
 'company': 'amazon'}

##### - Store

In [67]:
chunks_max = 15

In [68]:
vectorstore_engine = 'Finbot-embedding-2'

In [69]:
embedding_model = OpenAIEmbeddings(deployment=vectorstore_engine)

In [70]:
try:
    vectorstore.delete_collection()
except (ValueError, NameError):
    pass
for _ in range(0, len(chunked_data), chunks_max):
    vectorstore = Chroma.from_documents(
        documents=chunked_data[_: _ + chunks_max], embedding=embedding_model
    )

{'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/amazon_investorcom_1.pdf',
 'company': 'amazon'}

In [72]:
len(vectorstore.get()['documents'])

17

In [73]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents'])

In [75]:
vectorstore.get()['metadatas']

[{'company': 'amazon',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/amazon_investorcom_1.pdf'},
 {'company': 'amazon',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/amazon_investorcom_1.pdf'},
 {'company': 'nvidia',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/nvidia_investorcom_1.pdf'},
 {'company': 'nvidia',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/nvidia_investorcom_1.pdf'},
 {'company': 'apple',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/apple_investorcom_1.pdf'},
 {'company': 'apple',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/apple_investorcom_1.pdf'},
 {'company': 'tesla',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/tesla_investorcom_1.pdf'},
 {'company': 'tesla',
  'source': '/Users/rachitjoshi/Documents/Finbot/notebooks/../data/pdfs/tesla_investorcom_1.pdf'},
 {'company': 'tesla',
  

##### - Retrieve

In [173]:
question = "tell me about the world in brief?"
docs = vectorstore.similarity_search_with_relevance_scores(question, k=7)
len(docs)

7

In [174]:
len(docs[0][0].page_content)

3996

In [175]:
# print(docs[1][0].page_content)

##### - Generate

In [176]:
llm_engine = 'finbot-gpt'
llm_model = 'text-davinci-002'
temperature = 0
search_type = 'mmr'
retrieval_kwargs = {'k': 5, 'lambda_mult': 0.5, 'fetch_k':10}
answer_max_tokens = 512
save_folder = '../data/pdfs/'
prompt_template_file = os.path.join(os.getcwd(), '../scripts/prompt_template.txt')
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []
langchain.debug=False

In [177]:
############

In [178]:
base_llm = AzureOpenAI(
    engine=llm_engine, 
    model_name=llm_model, 
    temperature=temperature, 
    max_tokens=answer_max_tokens
)

In [179]:
langchain.debug=False

In [180]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [181]:
condense_question_template = os.path.join(
    os.getcwd(), '..', 'scripts/condense_question_template.txt'
)
condense_question_input = c.condense_question_input

In [182]:
chat_prompt = PromptTemplate.from_file(
    condense_question_template,
    input_variables=condense_question_input
)

In [183]:
qna_prompt_template = os.path.join(os.getcwd(), '..', 'scripts/qna_prompt_template.txt')
qna_prompt_input = c.qna_prompt_input
prompt_role = c.prompt_role

In [184]:
qna_prompt = PromptTemplate.from_file(
    qna_prompt_template,
    input_variables=qna_prompt_input,
    partial_variables={'role':prompt_role}
)

In [185]:
history_tokens = 2000
memory = ConversationSummaryBufferMemory(
    llm=base_llm, 
    memory_key="chat_history", 
    return_messages=True, 
    max_token_limit=history_tokens
)

In [186]:
qna_chain = ConversationalRetrievalChain.from_llm(
    base_llm, 
    retriever=vectorstore.as_retriever(
        search_type=search_type, search_kwargs=retrieval_kwargs
    ),
    memory=memory,
    condense_question_prompt=chat_prompt,
    combine_docs_chain_kwargs=dict(prompt=qna_prompt)
)

In [187]:
question = "tell me about the world in brief?"
results = qna_chain({"question": question})

In [188]:
print(results['answer'])

 Kevin McCarthy was ousted as speaker of America’s House of Representatives in a vote forced by Matt Gaetz, a fellow Republican congressman. The motion to unseat Mr McCarthy was supported by eight Republicans and all 208 Democrats present. Mr Gaetz, a firebrand from Florida, filed the “motion to vacate” on Monday after Mr McCarthy pushed through a bipartisan spending bill. The House has taken such a vote only once before, in 1910; that effort, however, was unsuccessful. Mr McCarthy’s exit has thrown the House into turmoil. The Republican majority has no obvious replacement for him. The judge overseeing Donald Trump’s civil trial for alleged business fraud imposed a gagging order to bar the former president, and others involved in the case, from making public remarks about court staff. He warned that violating the order would result in “serious sanctions”. Mr Trump had denigrated a law clerk on social media; the judge then ordered that his post be deleted. It was removed from Mr Trump’s